## Cash Liquidity Forecast
For the Data Product Cash Flow we want to expand the data product by calculating for upcoming periods the cash flow. This notebook shows an example workflow for the enrichment of the CashFlow data product which is going to be exposed back to SAP Datasphere in Business Data Cloud (BDC).
This involves in total the following steps for the overall prediction:
- Consume exposed data product over the Delta Share
- Prepare data for time series forecasting
- Perform hyperparameter optimization for time series prediction with model selection
- Log best model to MLflow

### Install packages
All necessary packages for this notebook are going to be outlined in the following notebook cell. In order to make sure that the results are reproducible, the following packages are going to be installed.
- Databricks feature engineering: Allows to store feature tables in Databricks

In [0]:
%pip install databricks-feature-engineering
%restart_python

### Import packages

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, date_trunc, sum, explode, sequence, min, max, lit, expr

### Setup Spark Session and consume data product
In order to isolate the created data assets, we create a catalog within Databricks and a respective schema within the catalog. Please replace the values <CATALOG_NAME> and <SCHEMA_NAME> with distinct values that provide you with a separated catalog and schema

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS <CATALOG_NAME>;
SET CATALOG <CATALOG_NAME>;
CREATE SCHEMA IF NOT EXISTS <SCHEMA_NAME>;
USE SCHEMA <SCHEMA_NAME>;

From BDC we expose a [delta enabled local table](https://help.sap.com/docs/SAP_DATASPHERE/c8a54ee704e94e15926551293243fd1d/154bdffb35814d5481d1f6de143a6b9e.html?locale=en-US) over the delta share which provides us with a table containing multiple entries for the same primary key. The dataset contains the __OPERATION_TYPE column marking the transactional statement (Insert, Update, Delete) together with the __TIMESTAMP column marking when this change happened. As we want to use the Cashflow transactional statements, we transform our dataset in the following to provide the most recent entry per primary key. In case the most recent entry is a deletion, we filter this record out.

In [0]:
spark = SparkSession.builder.appName("cash_flow_data_preparation").getOrCreate()
data = spark.read.table(<SHARE_TABLE_PATH>)
data = data.alias("l").\
    groupBy(col("CashFlowID")).agg(max(col("__TIMESTAMP")).alias("left__TIMESTAMP")).\
    join(
        data.alias("r"),
        (col("l.CashFlowID") == col("r.CashFlowID")) & (col("left__TIMESTAMP") == col("r.__TIMESTAMP")),
        "left"
    ).\
    where("'__OPERATION_TYPE' != 'D'").\
    select("r.*")

In [ ]:
data.show()

### Data preparation
For the data preparation of the Cash Flow data product for the time series forecast, we remodel the data by performing the following steps. The date column is going to be the posting date as the posting marks whether a Cash flow is booked or not. The forecast is performed on a monthly date sequence on the posting date. [See details under term definition for posting date](https://help.sap.com/glossary/?locale=en-US&term=posting%2520date):
1. Replace empty strings with Null values
2. Select necessary columns and filter out on the Posting date invalid dates and Null values
3. Floor Posting date column to month and rename date and value column
4. Group data on date column and sum up Cash Flow per month
5. Generate continuous time series range between minimum date and maximum date present in data
6. Join generated time sequence to time series data in order to provide continuous time series dataframe
7. Fill Null values with 0 as at those days no cash flow was recorded
8. Convert Spark dataframe to pandas dataframe

In [0]:
data = data.replace('', None)
data = data.select("CashFlowID", "PostingDate", "AmountInCompanyCodeCurrency").where("PostingDate != '9999-12-31' or PostingDate IS NOT NULL")

In [0]:
# Floor date and rename columns
data = data.\
    withColumn("PostingDate", date_trunc("month", col("PostingDate")).cast("date")).\
    withColumnsRenamed({"PostingDate": "ds", "Company_Code": "CompanyCode", "AmountInCompanyCodeCurrency": "y"})
# aggregate time series on date and sum cash flow
time_series_data = data.\
    select("ds", "CompanyCode", "y").\
    groupBy("ds", "CompanyCode").\
    agg(sum("y").alias("y")).\
    orderBy("ds")
# generate continous time series sequence
date_sequence_data = time_series_data\
    .select(
        explode(
            expr("sequence(min(ds), max(ds), INTERVAL 1 MONTH)")
            ).alias("ds"))
date_company_combination = time_series_data.select("CompanyCode").\
    distinct().\
    join(date_sequence_data, how="cross")
# join time series data together with time series sequence
time_series_data = time_series_data.\
    join(date_company_combination, on=["ds", "CompanyCode"], how="right").\
    fillna(0, subset=["y"])

#### Store prepared dataset to the Feature Store of Databricks
In order to reuse the dataset for our Training as well our Prediction, we store the transformed dataset into the feature store Databricks. This provides the possibility to not repeat the same data preparation script for both the Training as well as the Prediction notebook

In [0]:
fe_client = FeatureEngineeringClient()

In [0]:
fe_client.create_table(
    name="prepared_cash_flow_time_series",
    primary_keys=["ds", "CompanyCode"],
    schema=time_series_data.schema,
    description="Prepared Cash Flow data product for time series forecasting"
)

In [0]:
fe_client.write_table(
    name="prepared_cash_flow_time_series",
    df=time_series_data,
    mode="merge"
)